In [ ]:

!pip install tensorflow==2.12.0
!pip install tensorflow-model-optimization

In [ ]:
#Make sure it is 2.12.0 before doing any work
import tensorflow as tf
print(tf.__version__)

2.12.0


In [ ]:
# prompt: navigate to a folder in my google drive My Drive > data > fall_data

import os
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/My Drive/Data/
!pwd


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Data
/content/drive/My Drive/Data


In [ ]:
import pandas as pd

# Load the four CSV files into dataframes without the index
df_sitting = pd.read_csv('sitting_data.csv', index_col=False)
df_standing = pd.read_csv('standing_data.csv', index_col=False)
df_walking = pd.read_csv('walking_data.csv', index_col=False)
df_falling = pd.read_csv('falling_data.csv', index_col=False)

# Specify the columns that contain gyro and accel data
gyro_accel_columns = ['Gyro_X', 'Gyro_Y', 'Gyro_Z', 'Accel_X', 'Accel_Y', 'Accel_Z']  # Replace with actual column names



In [ ]:
# Import libraries (add libraries you need here)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout,Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# Function to create a sliding window dataframe for specific columns
def create_sliding_window_df(df, columns, window_size):
    sliding_window_data = []
    for i in range(len(df) - window_size + 1):
        window = df[columns].iloc[i:i + window_size].values.flatten()
        sliding_window_data.append(window)
    return pd.DataFrame(sliding_window_data)

In [ ]:
# Define your window_size
window_size = 15

# Create new dataframes with sliding windows for gyro and accel data
df_sitting_windowed = create_sliding_window_df(df_sitting, gyro_accel_columns,window_size)
df_standing_windowed = create_sliding_window_df(df_standing, gyro_accel_columns,window_size)
df_walking_windowed = create_sliding_window_df(df_walking, gyro_accel_columns,window_size)
df_falling_windowed = create_sliding_window_df(df_falling, gyro_accel_columns,window_size)

scaler = StandardScaler()
df_sitting_windowed = pd.DataFrame(scaler.fit_transform(df_sitting_windowed), columns=df_sitting_windowed.columns)
df_standing_windowed = pd.DataFrame(scaler.fit_transform(df_standing_windowed), columns=df_standing_windowed.columns)
df_walking_windowed = pd.DataFrame(scaler.fit_transform(df_walking_windowed), columns=df_walking_windowed.columns)
df_falling_windowed = pd.DataFrame(scaler.fit_transform(df_falling_windowed), columns=df_falling_windowed.columns)

# Add labels to the windowed dataframes
df_sitting_windowed['Label'] = 0
df_standing_windowed['Label'] = 1
df_walking_windowed['Label'] = 2
df_falling_windowed['Label'] = 3

In [ ]:
# DATAFRAME
# Concatenate all windowed dataframes into one large dataframe X
X_df = pd.concat([df_sitting_windowed, df_standing_windowed, df_walking_windowed, df_falling_windowed], ignore_index=True)

# Randomize the order of the X data
X_df = shuffle(X_df, random_state=42)
# Separate the labels into a dataframe y
y_df = X_df.pop('Label')

label_encoder = LabelEncoder()
y_encoded_df = label_encoder.fit_transform(y_df)
y_categorical_df = to_categorical(y_encoded_df, num_classes=4)  # Ensure 4 classes for one-hot encoding


In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, Input
from tensorflow.keras.utils import to_categorical
from tensorflow_model_optimization.quantization.keras import quantize_annotate_layer
from tensorflow_model_optimization.quantization.keras import quantize_annotate_model
from tensorflow_model_optimization.quantization.keras import quantize_scope
from tensorflow_model_optimization.quantization.keras import quantize_apply


In [ ]:
# DATAFRAME: DO NOT EXECUTE THIS YET
# Reshape X to be 3D [samples, window size, features]
n_features = 6 #X_array.shape[1] #features
X_reshape_df = X_df.to_numpy().reshape(X_df.shape[0], window_size, n_features) #Reshape to 3D array


# Split the data into training, testing, and validation sets
X_train_df, X_temp_df, y_train_df, y_temp_df = train_test_split(X_reshape_df, y_categorical_df, test_size=0.4, random_state=42)
X_test_df, X_val_df, y_test_df, y_val_df = train_test_split(X_temp_df, y_temp_df, test_size=0.5, random_state=42)

print("Data has been successfully split into training, testing, and validation sets.")
print(f"Training set size: {len(X_train_df)}")
print(f"Testing set size: {len(X_test_df)}")
print(f"Validation set size: {len(X_val_df)}")

input_shape = (window_size,n_features)  # Example: 10 features

# Build the CNN model
model = Sequential()
model.add(Input(shape=input_shape))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))  # Assuming 4 classes: sitting, standing, walking, falling

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_df = model.fit(X_train_df, y_train_df, epochs=10, batch_size=32, validation_data=(X_val_df, y_val_df))

# Evaluate the model
loss_df, accuracy_df = model.evaluate(X_test_df, y_test_df)
print(f'Test Accuracy: {accuracy_df:.2f}')

Data has been successfully split into training, testing, and validation sets.
Training set size: 6686
Testing set size: 2229
Validation set size: 2229
Epoch 1/10
209/209 [==============================] - 4s 8ms/step - loss: 1.2776 - accuracy: 0.4233 - val_loss: 0.9922 - val_accuracy: 0.6528
Epoch 2/10
209/209 [==============================] - 2s 7ms/step - loss: 0.9429 - accuracy: 0.6339 - val_loss: 0.6427 - val_accuracy: 0.7730
Epoch 3/10
209/209 [==============================] - 1s 7ms/step - loss: 0.7097 - accuracy: 0.7438 - val_loss: 0.4554 - val_accuracy: 0.8430
Epoch 4/10
209/209 [==============================] - 1s 4ms/step - loss: 0.5300 - accuracy: 0.8095 - val_loss: 0.3082 - val_accuracy: 0.9022
Epoch 5/10
209/209 [==============================] - 1s 4ms/step - loss: 0.4241 - accuracy: 0.8521 - val_loss: 0.2262 - val_accuracy: 0.9197
Epoch 6/10
209/209 [==============================] - 1s 4ms/step - loss: 0.3233 - accuracy: 0.8847 - val_loss: 0.1715 - val_accuracy: 0.95

In [ ]:
#Set-up
model = model
model_file_name = 'Check_Point.tflite' #change the name for your model

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Built-in operations - ReLU, Conv2D, MaxPool, Dense, etc...
    tf.lite.OpsSet.SELECT_TF_OPS     # Select TensorFlow operations - maybe for LSTM/RNN layers
]
converter.optimizations = [tf.lite.Optimize.DEFAULT] # See below
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open(model_file_name, 'wb') as f:
    f.write(tflite_model)

print("Model converted to TensorFlow Lite and saved as ", model_file_name)


Model converted to TensorFlow Lite and saved as  Check_Point.tflite
